# N2 — SPARC small-sample shared-\u03b5 MCMC\n
\n
對應 C4/C5：以小樣本（預設玩具資料）跑 **共享** $\varepsilon$ 的 MCMC，產出 $p(\varepsilon)$、每星系 $\chi^2_\nu$ 與 AIC/BIC。\n
可用 `data/sparc.csv`（由 `scripts/fetch_sparc.sh` 取得）或自動 fallback 至玩具資料。

In [ ]:
import os, json, numpy as np, matplotlib.pyplot as plt
os.makedirs("results", exist_ok=True)
np.random.seed(123)

from ptqg.data_sparc import load_sparc_csv, toy_sample
from ptqg.likelihood import loglike_gaussian, loglike_student_t, model_velocity_kms
from ptqg.mcmc import sample, gaussian, uniform
from ptqg.model_compare import aic_bic

# H0 預設 70 km/s/Mpc -> s^-1
# --- Manifest loader (shared across notebooks) ---
from ptqg.config import load_manifest, H0_si, annotate_string
CFG = load_manifest("manifest.yaml")   # 路徑可依需要改
H0 = H0_si(CFG)
ANN = annotate_string(CFG)
print("[cfg] H0 = %.3e s^-1, run:", H0, CFG["project"]["run_id"])
CSV = "data/sparc.csv"

if os.path.exists(CSV):
    galaxies = load_sparc_csv(CSV)
    print(f"[N2] Loaded CSV: {CSV} | #gal={len(galaxies)}")
else:
    galaxies = toy_sample(n_gal=10, seed=123)
    print(f"[N2] Using toy sample | #gal={len(galaxies)}")

# Priors: 可切換 planck / flat，此處示範 flat
prior_eps = uniform(0.0, 4.0)
prior_ups = gaussian(0.5, 0.1)

# MCMC 參數（預設小步數做 demo；完整請加大）
WALKERS, STEPS, SEED = 24, 1500, 123
out = sample(galaxies=galaxies, H0=H0, like="gaussian", prior_eps=prior_eps,
             prior_ups=prior_ups, n_walkers=WALKERS, n_steps=STEPS, seed=SEED)

eps_chain = out["eps_chain"]      # [steps, walkers]
logprob   = out["logprob"]
np.save("results/eps_chain.npy", eps_chain)
np.save("results/logprob.npy", logprob)

smax, wmax = np.unravel_index(np.nanargmax(logprob), logprob.shape)
eps_map = float(eps_chain[smax, wmax])
ups_map = {}
for g in galaxies:
    ups_map[g.name] = float(out["ups_chains"][g.name][smax, wmax])
eps_map

## 後驗圖與 MAP 速度剖面一例

In [ ]:
plt.figure(figsize=(6,4))
plt.hist(eps_chain.reshape(-1), bins=60, density=True, alpha=0.75)
plt.axvline(eps_map, color='k', lw=1, label=f"MAP={eps_map:.3f}")
plt.xlabel(r"$\varepsilon$"); plt.ylabel("Posterior density")
plt.legend(); plt.tight_layout(); plt.savefig("results/N2_eps_posterior.png", dpi=150)
plt.close()

gal0 = galaxies[0]
r_kpc = gal0.r_m / 3.085677581e19
vmod0 = model_velocity_kms(gal0, eps_map, H0, ups_map[gal0.name])
plt.figure(figsize=(6,4))
plt.errorbar(r_kpc, gal0.v_obs_kms, yerr=gal0.v_err_kms, fmt='o', ms=3, label='obs')
plt.plot(r_kpc, vmod0, lw=2, label='model')
plt.xlabel('r [kpc]'); plt.ylabel('v [km/s]')
plt.title(f"Example RC fit: {gal0.name}")
plt.legend(); plt.tight_layout(); plt.savefig("results/N2_example_rc.png", dpi=150)
plt.close()

## AIC/BIC 與每星系 $\chi^2_\nu$（以 Gaussian logL 計算）

In [ ]:
import csv
logL_map, chi2nu = loglike_gaussian(galaxies, eps_map, H0, ups_map)
k = 1 + len(galaxies)
Ntot = sum(g.r_m.size for g in galaxies)
AIC, BIC = aic_bic(logL_map, k, Ntot)
print(f"[N2] MAP eps={eps_map:.4f}, logL={logL_map:.2f}, AIC={AIC:.2f}, BIC={BIC:.2f}")

with open("results/N2_per_gal_chi2.csv", "w", newline="") as f:
    w = csv.writer(f); w.writerow(["name", "chi2_nu"])
    for n,v in chi2nu.items(): w.writerow([n, f"{v:.6f}"])

with open("results/N2_aic_bic.csv", "w", newline="") as f:
    w = csv.writer(f); w.writerow(["logL_max","k","Ntot","AIC","BIC"])
    w.writerow([f"{logL_map:.6f}", k, Ntot, f"{AIC:.6f}", f"{BIC:.6f}"])

# 摘要 JSON
summary = dict(eps_map=eps_map, logL_map=logL_map, AIC=AIC, BIC=BIC,
               walkers=WALKERS, steps=STEPS, seed=SEED, like='gaussian', H0=H0,
               Ntot=Ntot, n_gal=len(galaxies))
with open("results/N2_summary.json", "w") as f:
    json.dump(summary, f, indent=2)
summary